### Valence model


In [ ]:
import torch
import torchaudio

In [ ]:
import os
import requests
import torch

import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
labels_file = "tracks_features.csv"
#labels_file = "/content/drive/MyDrive/tracks_features.csv"
all_tracks = pd.read_csv(labels_file)

In [8]:
import torch

class AudioDataset(torch.utils.data.Dataset):
  def __init__(self, mfcc_tensor, df):
    self.df = df
    self.mfcc = mfcc_tensor
    self.mean = self.mfcc.mean()
    self.std = self.mfcc.std()

    # Standardize MFCC tensor
    self.mfcc = (self.mfcc - self.mean) / self.std

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    label = torch.tensor(self.df.iloc[idx]['valence'], dtype=torch.float32)
    mfcc = self.mfcc[idx]
    return mfcc, label

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

loaded_mfcc_tensor = torch.load("C://users/khala/Downloads/mfcc_tensor (1).pt")

# Split all_tracks and the loaded MFCC tensor
train_tracks, test_val_tracks = train_test_split(all_tracks, test_size=0.3, random_state=42)
test_tracks, val_tracks = train_test_split(test_val_tracks, test_size=0.5, random_state=42)

train_mfcc, test_val_mfcc = train_test_split(loaded_mfcc_tensor, test_size=0.3, random_state=42)
test_mfcc, val_mfcc = train_test_split(test_val_mfcc, test_size=0.5, random_state=42)

# Create the datasets
train_dataset = AudioDataset(train_mfcc, train_tracks)
val_dataset = AudioDataset(val_mfcc, val_tracks)
test_dataset = AudioDataset(test_mfcc, test_tracks)

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [46]:
import torch.nn as nn

# 2D CNN Model
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
    self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
    self.fc1 = nn.Linear(32 * 5 * 40, 64)
    self.fc2 = nn.Linear(64, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = nn.functional.relu(self.conv1(x))
    x = nn.functional.max_pool2d(x, 2)
    x = nn.functional.relu(self.conv2(x))
    x = nn.functional.max_pool2d(x, 2)
    x = x.view(x.size(0), -1)
    x = nn.functional.relu(self.fc1(x))
    x = self.fc2(x)

    x = self.sigmoid(x)
    
    # Multiply by 1000, round, and then divide by 1000 for precision purposes
    # x = torch.round(x * 1000) / 1000 # prevents loss from doing down

    return x

model = CNNModel().cuda()

In [47]:
import torch.optim as optim
from tqdm import tqdm # progress bar stuff
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

num_epochs = 20
for epoch in range(num_epochs):
  print(f"working on epoch: {epoch}")
  model.train()
  running_loss = 0.0
  for i, (inputs, labels) in enumerate(tqdm(train_loader)):
    inputs, labels = inputs.to(device), labels.to(device)
    #inputs = inputs.permute(1,0,2)
    #inputs = inputs.unsqueeze(1)
    #inputs = inputs.permute(0,1,2,3)
    
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs.squeeze(), labels)
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()

  epoch_loss = running_loss / (i + 1)
  print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {epoch_loss:.4f}")

  # Validation
  model.eval()
  val_running_loss = 0.0
  with torch.no_grad():
    for inputs, labels in val_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      #inputs = inputs.unsqueeze(1)
      #inputs = inputs.permute(0,1,2,3)
      outputs = model(inputs)
      loss = criterion(outputs.squeeze(), labels)
      val_running_loss += loss.item()

  val_epoch_loss = val_running_loss / len(val_loader)
  print(f"Validation Loss: {val_epoch_loss:.4f}")

working on epoch: 0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467/467 [00:04<00:00, 97.92it/s]


Epoch [1/20] - Loss: 0.0541
Validation Loss: 0.0449
working on epoch: 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467/467 [00:04<00:00, 99.87it/s]


Epoch [2/20] - Loss: 0.0439
Validation Loss: 0.0414
working on epoch: 2


...

Epoch [18/20] - Loss: 0.0238
Validation Loss: 0.0436
working on epoch: 18


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467/467 [00:04<00:00, 100.94it/s]


Epoch [19/20] - Loss: 0.0229
Validation Loss: 0.0422
working on epoch: 19


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 467/467 [00:04<00:00, 101.13it/s]


Epoch [20/20] - Loss: 0.0227
Validation Loss: 0.0413


In [48]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

true_labels = []
predicted_labels = []

model.eval()
with torch.no_grad():
  for inputs, labels in test_loader:
    inputs = inputs.unsqueeze(1).to(device)
    labels = labels.to(device)

    outputs = model(inputs.squeeze(1))

    true_labels.extend(labels.cpu().numpy())
    predicted_labels.extend(outputs.squeeze().cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

mse = mean_squared_error(true_labels, predicted_labels)
mae = mean_absolute_error(true_labels, predicted_labels)
rmse = np.sqrt(mse)
r2 = r2_score(true_labels, predicted_labels)
pearson_corr, p_value = pearsonr(true_labels, predicted_labels)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")
print(f"Pearson's Correlation Coefficient: {pearson_corr:.4f}")
print(f"P-value: {p_value:.4f}")

Mean Squared Error: 0.0414
Mean Absolute Error: 0.1609
Root Mean Squared Error: 0.2035
R-squared: 0.3342
Pearson's Correlation Coefficient: 0.6052
P-value: 0.0000
